# Purpose:
- Build dataframe of whisker features per trial
- To calculate angle discrimination using whisker features, changes of sensory input across training, etc.
- Per trial, take mean of whisker features across all touches, except for touch count
- Save for pre-answer touch, post-answer touch, all touch
    - pre-answer touch: touches that ended before the answer time
    - post-answer touch: touches that started after the answer time

In [9]:
import numpy as np
import pandas as pd
from pathlib import Path
import glob
from tqdm.notebook import tqdm
w_dir = Path('E:\TPM\JK\WhiskerVideo')
b_dir = Path('E:\TPM\JK\SoloData')
mice =          [25,    27,   30,   36,     37,     38,     39,     41,     52,     53,     54,     56]


In [4]:
mouse = mice[0]
session_folder_list = glob.glob(str(w_dir / f'JK{mouse:03d}S*'))
session_folder = session_folder_list[0]
session_name = session_folder.split('\\')[-1]
wf_fn = w_dir / session_name / f'{session_name}_whisker_final_h5.mat'
whisker_pkl_fn = w_dir / session_name / f'{session_name}_whisker.pkl'
whisker_df = pd.read_pickle(whisker_pkl_fn)

In [6]:
whisker_df.columns

Index(['amplitude', 'arcLength', 'kappaH', 'kappaV', 'midpoint', 'phase',
       'phi', 'poleAngle', 'poleDistance', 'poleMovingFrames', 'polePosition',
       'poleUpFrames', 'protractionSlide', 'protractionSlideByWhisking',
       'protractionTFchunks', 'protractionTFchunksByWhisking',
       'protractionTouchDuration', 'protractionTouchDurationByWhisking',
       'retractionSlide', 'retractionSlideByWhisking', 'retractionTFchunks',
       'retractionTFchunksByWhisking', 'retractionTouchDuration',
       'retractionTouchDurationByWhisking', 'theta', 'whisker_time',
       'whiskingStartFrames', 'mouse_name', 'session_name'],
      dtype='object')

In [8]:
trialNums = whisker_df.index.values
ti = 10
trialNum = trialNums[ti]
touch_count = len(whisker_df.loc[trialNum, 'protractionTFchunksByWhisking'])



[array([607., 608., 609., 610.]),
 array([641., 642., 643., 644.]),
 array([653., 654., 655., 656., 657., 658., 659.])]

In [ ]:
for mouse in tqdm(mice):
    session_folder_list = glob.glob(str(w_dir / f'JK{mouse:03d}S*'))
    for session_folder in tqdm(session_folder_list):
        session_name = session_folder.split('\\')[-1]
        wf_fn = w_dir / session_name / f'{session_name}_whisker_final_h5.mat'
        whisker_pkl_fn = w_dir / session_name / f'{session_name}_whisker.pkl'